## Import training and testing data

### Import dependencies

In [1]:
import pandas as pd
import numpy as np

### Import data from S3 bucket

In [2]:
df = pd.read_csv('https://aws-matchmaker-bucket.s3-us-west-2.amazonaws.com/SpeedDatingPreprocessed.csv')

In [3]:
df.head()

,couple_id,f_age,f_imprace,f_attr,f_sinc,f_intel,f_fun,f_amb,f_race,f_intrace,...,m_imprace,m_attr,m_sinc,m_intel,m_fun,m_amb,m_race,samerace,match,m_intrace
0,f1-m11,21.0,2.0,6.0,9.0,7.0,7.0,6.0,4.0,0.0,...,7.0,6.0,8.0,8.0,8.0,8.0,2.0,0,0,0.0
1,f1-m12,21.0,2.0,7.0,8.0,7.0,8.0,5.0,4.0,0.0,...,1.0,7.0,8.0,10.0,7.0,7.0,2.0,0,0,0.0
2,f1-m13,21.0,2.0,5.0,8.0,9.0,8.0,5.0,4.0,2.0,...,3.0,10.0,10.0,10.0,10.0,10.0,4.0,1,1,3.0
3,f1-m14,21.0,2.0,7.0,6.0,8.0,7.0,6.0,4.0,0.0,...,1.0,7.0,8.0,9.0,8.0,9.0,2.0,0,1,0.0
4,f1-m15,21.0,2.0,5.0,6.0,7.0,7.0,6.0,4.0,0.0,...,3.0,8.0,7.0,9.0,6.0,9.0,3.0,0,1,0.0


Prepare to train and test models: split data into features, X, and classes, y.

In [4]:
X = df.drop(["couple_id","match"], axis=1)
y = df["match"]
print(X.shape, y.shape)

(3381, 19) (3381,)


Split data into a set for training and a smaller set for testing.

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Evaluate various models

### Logistic Regression

In [6]:
from sklearn.linear_model import LogisticRegression

Create a logistic regression classification model and fit it to the training data.

In [7]:
classifier = LogisticRegression(solver='newton-cg')
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

Review scores, confusion matrix, and classification report.

In [8]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8445759368836292
Testing Data Score: 0.8557919621749409


In [9]:
predictions = classifier.predict(X_test)

In [10]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"tn: {tn:3d}   fp: {fp:3d}")
print(f"fn: {fn:3d}   tp: {tp:3d}")

tn: 684   fp:  21
fn: 101   tp:  40


In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92       705
           1       0.66      0.28      0.40       141

   micro avg       0.86      0.86      0.86       846
   macro avg       0.76      0.63      0.66       846
weighted avg       0.84      0.86      0.83       846



### K Nearest Neighbors

Create and fit several different K Nearest Neighbor models.

In [12]:
from sklearn.neighbors import KNeighborsClassifier

train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")

k: 1, Train/Test Score: 1.000/0.751
k: 3, Train/Test Score: 0.891/0.807
k: 5, Train/Test Score: 0.871/0.822
k: 7, Train/Test Score: 0.865/0.851
k: 9, Train/Test Score: 0.855/0.848
k: 11, Train/Test Score: 0.852/0.846
k: 13, Train/Test Score: 0.848/0.849
k: 15, Train/Test Score: 0.848/0.850
k: 17, Train/Test Score: 0.845/0.851
k: 19, Train/Test Score: 0.842/0.848


The best models appear to be K = 15 and 17.

In [13]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=15, p=2,
           weights='uniform')

Review scores, confusion matrix, and classification report.

In [14]:
print(f"Training Data Score: {knn.score(X_train, y_train)}")
print(f"Testing Data Score: {knn.score(X_test, y_test)}")

Training Data Score: 0.8481262327416174
Testing Data Score: 0.8498817966903073


In [15]:
predictions = knn.predict(X_test)

In [16]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"tn: {tn:3d}   fp: {fp:3d}")
print(f"fn: {fn:3d}   tp: {tp:3d}")

tn: 691   fp:  14
fn: 113   tp:  28


In [17]:
print(classification_report(y_test, predictions,target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92       705
           1       0.67      0.20      0.31       141

   micro avg       0.85      0.85      0.85       846
   macro avg       0.76      0.59      0.61       846
weighted avg       0.83      0.85      0.81       846



### Decision Tree

Create and fit a decision tree model.

In [18]:
from sklearn import tree

dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(X_train, y_train)

Review scores, confusion matrix, and classification report.

In [19]:
print(f"Training Data Score: {dtc.score(X_train, y_train)}")
print(f"Testing Data Score: {dtc.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.7565011820330969


In [20]:
predictions = dtc.predict(X_test)

In [21]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"tn: {tn:3d}   fp: {fp:3d}")
print(f"fn: {fn:3d}   tp: {tp:3d}")

tn: 588   fp: 117
fn:  89   tp:  52


In [22]:
print(classification_report(y_test, predictions,target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85       705
           1       0.31      0.37      0.34       141

   micro avg       0.76      0.76      0.76       846
   macro avg       0.59      0.60      0.59       846
weighted avg       0.78      0.76      0.77       846



### Random Forest

Create and fit a random forest model.

In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=300)
rf = rf.fit(X_train, y_train)

Review scores, confusion matrix, and classification report.

In [24]:
print(f"Training Data Score: {rf.score(X_train, y_train)}")
print(f"Testing Data Score: {rf.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8522458628841607


In [25]:
predictions = rf.predict(X_test)

In [26]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"tn: {tn:3d}   fp: {fp:3d}")
print(f"fn: {fn:3d}   tp: {tp:3d}")

tn: 689   fp:  16
fn: 109   tp:  32


In [27]:
print(classification_report(y_test, predictions,target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92       705
           1       0.67      0.23      0.34       141

   micro avg       0.85      0.85      0.85       846
   macro avg       0.77      0.60      0.63       846
weighted avg       0.83      0.85      0.82       846



### Support Vector Machine

Create and fit a support vector machine model.

In [28]:
from sklearn.svm import SVC 

svcModel = SVC(kernel='linear',class_weight='balanced')
svcModel.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

Review scores, confusion matrix, and classification report.

In [29]:
print(f"Training Data Score: {svcModel.score(X_train, y_train)}")
print(f"Testing Data Score: {svcModel.score(X_test, y_test)}")

Training Data Score: 0.7400394477317555
Testing Data Score: 0.7092198581560284


In [30]:
predictions = svcModel.predict(X_test)

In [31]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"tn: {tn:3d}   fp: {fp:3d}")
print(f"fn: {fn:3d}   tp: {tp:3d}")

tn: 499   fp: 206
fn:  40   tp: 101


In [32]:
print(classification_report(y_test, predictions,target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.93      0.71      0.80       705
           1       0.33      0.72      0.45       141

   micro avg       0.71      0.71      0.71       846
   macro avg       0.63      0.71      0.63       846
weighted avg       0.83      0.71      0.74       846



### Deep Learning

Create and fit a deep learning/neural network model.

In [33]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=10, activation='relu', input_dim=19))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

Using TensorFlow backend.


In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                200       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 22        
Total params: 662
Trainable params: 662
Non-trainable params: 0
________________________________________________________

In [35]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
y = np.array(y).reshape(-1,1)
print(X.shape, y.shape)

(3381, 19) (3381, 1)


In [37]:
model.fit(
    X_train,
    y_train,
    epochs=200,
    shuffle=True,
    verbose=1
)

Epoch 1/200
2535/2535 [==============================] - 0s 80us/step - loss: 0.4660 - accuracy: 0.8256
Epoch 2/200
2535/2535 [==============================] - 0s 25us/step - loss: 0.4503 - accuracy: 0.8256
Epoch 3/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.4417 - accuracy: 0.8256
Epoch 4/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.4251 - accuracy: 0.8256
Epoch 5/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.4150 - accuracy: 0.8256
Epoch 6/200
2535/2535 [==============================] - 0s 26us/step - loss: 0.4086 - accuracy: 0.8252
Epoch 7/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.4055 - accuracy: 0.8256
Epoch 8/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.4018 - accuracy: 0.8264
Epoch 9/200
2535/2535 [==============================] - 0s 25us/step - loss: 0.4012 - accuracy: 0.8256
Epoch 10/200
2535/2535 [==============================] - 0s 24u

2535/2535 [==============================] - 0s 24us/step - loss: 0.3683 - accuracy: 0.8454
Epoch 80/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.3638 - accuracy: 0.8442
Epoch 81/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.3634 - accuracy: 0.8465
Epoch 82/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.3630 - accuracy: 0.8454
Epoch 83/200
2535/2535 [==============================] - 0s 25us/step - loss: 0.3650 - accuracy: 0.8442
Epoch 84/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.3627 - accuracy: 0.8458
Epoch 85/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.3649 - accuracy: 0.8454
Epoch 86/200
2535/2535 [==============================] - 0s 25us/step - loss: 0.3619 - accuracy: 0.8442
Epoch 87/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.3648 - accuracy: 0.8442
Epoch 88/200
2535/2535 [==============================] - 0s 24us/st

2535/2535 [==============================] - 0s 23us/step - loss: 0.3647 - accuracy: 0.8391
Epoch 157/200
2535/2535 [==============================] - 0s 22us/step - loss: 0.3560 - accuracy: 0.8465
Epoch 158/200
2535/2535 [==============================] - 0s 22us/step - loss: 0.3552 - accuracy: 0.8497
Epoch 159/200
2535/2535 [==============================] - 0s 23us/step - loss: 0.3580 - accuracy: 0.8454
Epoch 160/200
2535/2535 [==============================] - 0s 23us/step - loss: 0.3568 - accuracy: 0.8430
Epoch 161/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.3649 - accuracy: 0.8477
Epoch 162/200
2535/2535 [==============================] - 0s 23us/step - loss: 0.3556 - accuracy: 0.8485
Epoch 163/200
2535/2535 [==============================] - 0s 24us/step - loss: 0.3573 - accuracy: 0.8481
Epoch 164/200
2535/2535 [==============================] - 0s 25us/step - loss: 0.3558 - accuracy: 0.8462
Epoch 165/200
2535/2535 [==============================] - 0

Review scores, confusion matrix, and classification report.

In [38]:
predictions = model.predict_classes(X_test)

In [39]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"tn: {tn:3d}   fp: {fp:3d}")
print(f"fn: {fn:3d}   tp: {tp:3d}")

tn: 687   fp:  18
fn: 106   tp:  35


In [40]:
print(classification_report(y_test, predictions,target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92       705
           1       0.66      0.25      0.36       141

   micro avg       0.85      0.85      0.85       846
   macro avg       0.76      0.61      0.64       846
weighted avg       0.83      0.85      0.82       846



## Conclusions

The better performing models all produce similar scores, confusion matrices, and classification reports, suggesting that our current limitations are more related to the data and the selected features than to the choice of models.